In [1]:
from copy import deepcopy
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
from operators_cython import perturbation, best_father, fitness

import numpy as np
import pandas as pd
import os, getopt
from scipy.spatial.distance import cdist
from math import inf, sqrt
from random import choice, seed, random
from sortedcollections import SortedDict
from time import perf_counter
from disjoint_set import DisjointSet

env = gp.Env(empty=True)
env.start()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-27


<gurobipy.Env, No parameter changes>

In [2]:
class tree:
    def __init__(self, n, Q, D):
        self.parent = np.ones(n, dtype = int) * -1
        self.gate = np.zeros(n, dtype = int)
        self.load = np.zeros(n, dtype = int)
        self.arrival = np.zeros(n)
        self.capacity = Q
        self.distance = D

    def connect(self, k, j):
        """k: parent, j: children"""
        if self.gate[k] == 0:
            self.gate[j] = gate = j
        else:
            self.gate[j] = gate = self.gate[k]
        self.load[gate] += 1
        self.parent[j] = k
        self.arrival[j] = self.arrival[k] + distance(k,j)
        if not self.arrival[j] >= earliest[j]:
            self.arrival[j] = earliest[j]
            
    def fitness(self):
        cost, feasible = 0, True 
        for j,k in enumerate(self.parent):
            if j != 0:
                cost += distance(k,j)
                arr, lat = self.arrival[j], latest[j]
                if  arr > lat:
                    feasible = False
                    cost += (arr - lat) * rho

        return cost, feasible
    
    def __repr__(self):
        
        parent = f'parent: [{", ".join([str(elem) for elem in self.parent])}]\n'
        gate = f'gate: [{", ".join([str(elem) for elem in self.gate])}]\n'
        load = f'load: [{", ".join([str(elem) for elem in self.load])}]\n'
        arrival = f'arrival: [{", ".join([str(elem) for elem in self.arrival])}]\n'
        return parent + gate + load + arrival
    
class instance():
    def __init__(self, name, capacity, node_data, num, reset_demand = True):
        self.name = name
        self.n = n = num + 1
        self.capacity = int(capacity)
        self.index, self.xcoords, self.ycoords, self.demands, self.earliest, self.latest\
            = extract_data(node_data[:n])

        self.nodes = np.array(range(n), dtype = int)
        self.edges = [(i,j) for i in self.nodes for j in self.nodes[1:] if i != j]

        #demands = 1 for all nodes 
        if reset_demand:
            self.demands = {i:1 for i in self.nodes}

        # cost = time = distance for simplicity
        global D
        aux = np.vstack((self.xcoords, self.ycoords)).T
        D  = cdist(aux,aux, metric='euclidean')

        self.cost = D
        self.maxcost = self.cost.mean()

    def dist(self,i,j):
        x = self.xcoords[i] - self.xcoords[j]
        y = self.ycoords[i] - self.ycoords[j]
        return sqrt(x**2 + y**2)

def distance(i,j):
    return D[i,j]

def initialize(ins):
    global D, Q, earliest, latest, xcoords, ycoords
    D = ins.cost
    Q = ins.capacity
    earliest, latest = ins.earliest, ins.latest
    xcoords, ycoords = ins.xcoords, ins.ycoords

def read_instance(location):
    node_data = []
    with open(location,"r") as inst:
        for i, line in enumerate(inst):
            if i in [1,2,3,5,6,7,8]:
                pass
            elif i == 0:
                name = line.strip()
            elif i == 4:
                capacity = line.strip().split()[-1]
            else:
                node_data.append(line.strip().split()[0:-1])
    return name, capacity, node_data

def extract_data(nodes):
    # Read txt solutions
    index, xcoords, ycoords, demands, earliest, latest = list(zip(*nodes))
        
    index = np.array([int(i) for i in index], dtype = int)
    xcoords = np.array([float(i) for i in xcoords])
    ycoords = np.array([float(i) for i in ycoords])
    demands = np.array([float(i) for i in demands])
    earliest = np.array([float(i) for i in earliest])
    latest = np.array([float(i) for i in latest])

    return index, xcoords, ycoords, demands, earliest, latest

def prim(ins, vis  = False, initial = False):
    
    initialize(ins)

    nodes, n = ins.nodes, ins.n
    start = perf_counter()

    s = tree(n, Q, D)
    itree = set() # muestra que es lo ultimo que se ha añadido
    nodes_left = set(nodes)

    d = inf
    for j in nodes[1:]:
        di = distance(0,j)
        if di < d:
            d = di
            v = j

    itree.add(0) #orden en que son nombrados
    itree.add(v)
    nodes_left.remove(0)
    nodes_left.remove(v)
    
    s.connect(0,v)
    cost = distance(0,v)

    while len(nodes_left) > 0:
        min_tree = inf
        for j in nodes_left:
            min_node = inf
            for ki in itree:# k: parent, j: offspring
                # calcula si alcanza a llegar desde alguno de los nodos que ya estan colocados
                dkj = distance(ki,j)
                # criterion = dkj
                tj = s.arrival[ki] + dkj
                Qj = s.load[s.gate[ki]]

                if tj <= latest[j] and Qj < Q: # isFeasible() # reescribir

                    if tj < earliest[j]:
                        tj = earliest[j]

                    crit_node = dkj
                    if crit_node < min_node:
                        min_node = crit_node
                        k = ki
                
            ### best of the node
            crit_tree = crit_node

            if crit_tree < min_tree:
                kk = k
                jj = j
                min_tree = crit_tree

        itree.add(jj)
        nodes_left.remove(jj)
        s.connect(kk,jj)
        cost += distance(kk,jj)

    time = perf_counter() - start
    
    if initial:
        return s , cost
        
    else:
        best_bound = None
        gap = None
        return cost, time, best_bound, gap

In [3]:
name, capacity, node_data = read_instance("gehring instances/1000/rc1_10_2.txt")
ins = instance(name, capacity, node_data, 100)
# ins.capacity = 10
initialize(ins)

s, cost = prim(ins, vis = False, initial = True)
initial_ = deepcopy(s)
print("prim:", cost)

prim: 5276.2005026094885


In [4]:
def gurobi_solution(ins, vis = False, time_limit = 1800, verbose = False, initial = False, start = None, rando = False):
    
    n = ins.n
    edges, cost = gp.multidict({(i,j): D[i,j] for (i,j) in ins.edges})
    nodes, earliest, latest, demands = gp.multidict({i: (ins.earliest[i], ins.latest[i], ins.demands[i]) for i in ins.nodes })
    nodesv = nodes[1:]

    M = max(latest.values()) + max(cost.values())

    # model and variables
    mdl = gp.Model(ins.name) if verbose else gp.Model(ins.name, env = env)
    
    if rando:
        x = gp.tupledict()
        for i, j in edges:
            ii = start.parent[j]
            if ii == i:
                if random() < 0.1:
                    x[i,j] = mdl.addVar(vtype = GRB.BINARY, lb=1, ub=1, name = "x[%d,%d]" % (i,j))
                else:
                    x[i,j] = mdl.addVar(vtype = GRB.BINARY, lb=0, ub=1, name = "x[%d,%d]" % (i,j))
            else:
                x[i,j] = mdl.addVar(vtype = GRB.BINARY, lb=0, ub=1, name = "x[%d,%d]" % (i,j))

    else:
        x = mdl.addVars(edges, vtype = GRB.BINARY, name = "x") #
    y = mdl.addVars(edges, vtype = GRB.CONTINUOUS, name = "y", lb = 0)
    d = mdl.addVars(nodes, vtype = GRB.CONTINUOUS, name = "d", lb = 0)

    if start is not None:
        for j in nodesv:
            i = start.parent[j]
            x[(i,j)].Start = 1
            d[j].Start = start.arrival[j]

    mdl.setObjective(x.prod(cost))

    R1 = mdl.addConstrs((gp.quicksum(x[(i,j)] for i in nodes if i!=j) == 1 for j in nodesv),name = "R1")
    R2 = mdl.addConstrs((gp.quicksum(y[(i,j)] for i in nodes if i!=j) - gp.quicksum(y[(j,i)] for i in nodesv if i!=j) == demands[j] for j in nodesv), name = "R2") 
    R3 = mdl.addConstrs((x[(i,j)] <= y[(i,j)] for i,j in edges),name = "R3") 
    R4 = mdl.addConstrs((y[(i,j)] <= Q * x[(i,j)] for i,j in edges), name = "R4") 
    R5 = mdl.addConstrs((d[i] + cost[(i,j)] - d[j] <= M * (1 - x[(i,j)]) for i,j in edges), name = "R5") 
    R6 = mdl.addConstrs((d[i] >= earliest[i] for i in nodes), name = "R6") 
    R7 = mdl.addConstrs((d[i] <= latest[i] for i in nodes), name = "R7")

    mdl.Params.TimeLimit = time_limit
    mdl.Params.Threads = 1

    solution = mdl.optimize()
    obj = mdl.getObjective()
    objective_value = obj.getValue()
    s = tree(n, Q, D)

    if not initial:

        time = mdl.Runtime
        best_bound = mdl.ObjBound
        gap = mdl.MIPGap

        return objective_value, time, best_bound, gap

    else: 
        departure = np.zeros(n)
        for i,j in edges:
            if x[(i,j)].X > 0.9:
                s.parent[j] = i
                departure[j] = d[j].X

        for j in sorted(nodes, key = lambda x: departure[x]):
            if j != 0:
                k = s.parent[j]
                s.connect(k,j)

        optimal = True if mdl.MIPGap < 0.0001 else False

        return s, objective_value, optimal

In [5]:
s, cost, optimal =gurobi_solution(ins, vis = False, time_limit= 5, verbose = False, initial=True, start = deepcopy(initial_))

Set parameter TimeLimit to value 5
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1240P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 1 threads

Optimize a model with 30402 rows, 20101 columns and 100102 nonzeros
Model fingerprint: 0x186e5263
Variable types: 10101 continuous, 10000 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]

User MIP start produced solution with objective 5276.2 (0.05s)
Loaded user MIP start with objective 5276.2

Presolve removed 17526 rows and 10332 columns
Presolve time: 0.80s
Presolved: 12876 rows, 9769 columns, 82572 nonzeros
Variable types: 4885 continuous, 4884 integer (4884 binary)

Root relaxation: objective 2.868134e+03, 9813 iterations, 1.00 seconds (0.90 work units)

    Nodes    | 

In [6]:
n = ins.n
edges, cost = gp.multidict({(i,j): D[i,j] for (i,j) in ins.edges})
nodes, earliest, latest, demands = gp.multidict({i: (ins.earliest[i], ins.latest[i], ins.demands[i]) for i in ins.nodes })
nodesv = nodes[1:]

M = max(latest.values()) + max(cost.values())
start = deepcopy(initial_)
rando = False

In [7]:
mdl = gp.Model(ins.name) if True else gp.Model(ins.name, env = env)

if rando:
    x = gp.tupledict()
    for i, j in edges:
        ii = start.parent[j]
        if ii == i:
            if random() < 0.1:
                x[i,j] = mdl.addVar(vtype = GRB.BINARY, lb=1, ub=1, name = "x[%d,%d]" % (i,j))
            else:
                x[i,j] = mdl.addVar(vtype = GRB.BINARY, lb=0, ub=1, name = "x[%d,%d]" % (i,j))
        else:
            x[i,j] = mdl.addVar(vtype = GRB.BINARY, lb=0, ub=1, name = "x[%d,%d]" % (i,j))

else:
    x = mdl.addVars(edges, vtype = GRB.BINARY, name = "x") #
y = mdl.addVars(edges, vtype = GRB.CONTINUOUS, name = "y", lb = 0)
d = mdl.addVars(nodes, vtype = GRB.CONTINUOUS, name = "d", lb = 0)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-27


In [8]:
if start is not None:
    for j in nodesv:
        i = start.parent[j]
        x[(i,j)].Start = 1
        d[j].Start = start.arrival[j]

In [9]:
mdl.Params.TimeLimit = 60 * 5
# mdl.Params.Threads = 1

solution = mdl.optimize()
obj = mdl.getObjective()
objective_value = obj.getValue()
s = tree(n, Q, D)

Set parameter TimeLimit to value 300
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i5-1240P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 0 rows, 20101 columns and 0 nonzeros
Model fingerprint: 0x8c060014
Variable types: 10101 continuous, 10000 integer (10000 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]



User MIP start produced solution with objective 0 (0.01s)
Loaded user MIP start with objective 0


Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [10]:
departure = np.zeros(n)
for i,j in edges:
    if x[(i,j)].X > 0.9:
        s.parent[j] = i
        departure[j] = d[j].X

for j in sorted(nodes, key = lambda x: departure[x]):
    if j != 0:
        k = s.parent[j]
        s.connect(k,j)

optimal = True if mdl.MIPGap < 0.0001 else False

Set parameter TimeLimit to value 5
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)



CPU model: 12th Gen Intel(R) Core(TM) i5-1240P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 1 threads

Optimize a model with 482 rows, 301 columns and 1454 nonzeros
Model fingerprint: 0xdc6b046a
Variable types: 157 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [4e+01, 6e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 225 rows and 116 columns
Presolve time: 0.01s
Presolved: 257 rows, 185 columns, 1209 nonzeros
Variable types: 99 continuous, 86 integer (86 binary)
Found heuristic solution: objective 1478.4719886

Root relaxation: objective 1.244228e+03, 30 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1244.22820    0    2 1478.47199 1244.22820  15.8%     -    0s
H  

(SortedDict({2: 67, 4: 8, 5: 0, 6: 0, 7: 6, 8: 2, 24: 67, 28: 7, 32: 6, 57: 5, 67: 0, 90: 0}),
 SortedDict({2: 67, 4: 67, 5: 5, 6: 6, 7: 6, 8: 67, 24: 67, 28: 6, 32: 6, 57: 5, 67: 67, 90: 90}),
 {2: 0,
  4: 0,
  5: 2,
  6: 4,
  7: 0,
  8: 0,
  24: 0,
  28: 0,
  32: 0,
  57: 0,
  67: 5,
  90: 1},
 SortedDict({0: 0, 2: 231.0, 4: 399.37943641267117, 5: 122.0, 6: 75.8023746329889, 7: 461.0, 8: 361.73637596323374, 24: 530.0, 28: 824.0, 32: 398.0, 57: 366.69777277286363, 67: 60.21627686929839, 90: 1192.0}))